In [1]:
import numpy as np
from scipy.optimize import line_search
from scipy.linalg import cho_factor, cho_solve

#1.1
def gradient_descent(oracle, x0, tolerance=1e-5, max_iter=1000,
                     line_search_options=None, trace=False):
    """
    Градиентный спуск с различными стратегиями выбора шага.
    """
    if line_search_options is None:
        line_search_options = {'method': 'Wolfe', 'c1': 1e-4, 'c2': 0.9}

    x_k = np.copy(x0)
    grad_0 = oracle.grad(x0)
    grad_norm_0_sq = np.dot(grad_0, grad_0)

    alpha = 1.0  # начальная длина шага

    if trace:
        history = {'time': [], 'func': [], 'grad_norm': [], 'x': []}

    for k in range(max_iter):
        # Вычисление градиента
        f_k = oracle.func(x_k)
        grad_k = oracle.grad(x_k)
        grad_norm_sq = np.dot(grad_k, grad_k)

        if trace:
            history['func'].append(f_k)
            history['grad_norm'].append(np.sqrt(grad_norm_sq))
            history['x'].append(np.copy(x_k))

        # Критерий остановки
        if grad_norm_sq <= tolerance * grad_norm_0_sq:
            message = f"success after {k} iterations, gradient norm {np.sqrt(grad_norm_sq):.2e}"
            return (x_k, message, history) if trace else (x_k, message)

        # Направление спуска (антиградиент)
        d_k = -grad_k

        # Линейный поиск
        method = line_search_options.get('method', 'Wolfe')

        if method == 'Wolfe':
            # Используем условия Вульфа
            def phi(alpha):
                return oracle.func(x_k + alpha * d_k)

            def phi_grad(alpha):
                return oracle.grad(x_k + alpha * d_k).dot(d_k)

            alpha, fc, gc, f_new, g_new, _ = line_search(
                phi, phi_grad, f_k, grad_k.dot(d_k),
                c1=line_search_options.get('c1', 1e-4),
                c2=line_search_options.get('c2', 0.9),
                maxiter=50
            )

            if alpha is None:
                # Если line_search не нашел шаг, используем бэктрекинг
                alpha = 1.0
                c = line_search_options.get('c1', 1e-4)
                while phi(alpha) > f_k + c * alpha * grad_k.dot(d_k):
                    alpha /= 2

        elif method == 'Armijo':
            # Условие Армихо с бэктрекингом
            alpha = line_search_options.get('initial_alpha', 1.0)
            c = line_search_options.get('c1', 1e-4)

            while oracle.func(x_k + alpha * d_k) > f_k + c * alpha * grad_k.dot(d_k):
                alpha /= 2

        elif method == 'Constant':
            # Постоянный шаг
            alpha = line_search_options.get('alpha', 0.01)

        # Обновление точки
        x_k = x_k + alpha * d_k

    message = f"iterations limit exceeded after {max_iter} iterations"
    return (x_k, message, history) if trace else (x_k, message)

#1.2
def newton(oracle, x0, tolerance=1e-5, max_iter=100,
           line_search_options=None, trace=False):
    """
    Метод Ньютона с различными стратегиями выбора шага.
    """
    if line_search_options is None:
        line_search_options = {'method': 'Wolfe', 'c1': 1e-4, 'c2': 0.9}

    x_k = np.copy(x0)
    grad_0 = oracle.grad(x0)
    grad_norm_0_sq = np.dot(grad_0, grad_0)

    if trace:
        history = {'time': [], 'func': [], 'grad_norm': [], 'x': []}

    for k in range(max_iter):
        # Вычисление градиента и гессиана
        f_k = oracle.func(x_k)
        grad_k = oracle.grad(x_k)
        hess_k = oracle.hess(x_k)

        grad_norm_sq = np.dot(grad_k, grad_k)

        if trace:
            history['func'].append(f_k)
            history['grad_norm'].append(np.sqrt(grad_norm_sq))
            history['x'].append(np.copy(x_k))

        # Критерий остановки
        if grad_norm_sq <= tolerance * grad_norm_0_sq:
            message = f"success after {k} iterations, gradient norm {np.sqrt(grad_norm_sq):.2e}"
            return (x_k, message, history) if trace else (x_k, message)

        try:
            # Решение системы Холецкого для нахождения направления Ньютона
            L, lower = cho_factor(hess_k)
            d_k = -cho_solve((L, lower), grad_k)
        except np.linalg.LinAlgError:
            # Если матрица не положительно определена, используем антиградиент
            d_k = -grad_k

        # Линейный поиск (всегда начинаем с alpha=1 для метода Ньютона)
        alpha = 1.0
        method = line_search_options.get('method', 'Wolfe')

        if method == 'Wolfe':
            # Условия Вульфа
            c1 = line_search_options.get('c1', 1e-4)
            c2 = line_search_options.get('c2', 0.9)

            def phi(a):
                return oracle.func(x_k + a * d_k)

            phi_0 = f_k
            phi_grad_0 = grad_k.dot(d_k)

            # Проверяем alpha=1 сначала
            phi_1 = phi(1.0)
            if phi_1 <= phi_0 + c1 * phi_grad_0:
                # Проверяем второе условие Вульфа
                grad_1 = oracle.grad(x_k + d_k)
                phi_grad_1 = grad_1.dot(d_k)
                if abs(phi_grad_1) <= c2 * abs(phi_grad_0):
                    alpha = 1.0
                else:
                    # Если второе условие не выполнено, ищем подходящий alpha
                    alpha = 0.5
                    while phi(alpha) > phi_0 + c1 * alpha * phi_grad_0:
                        alpha /= 2
            else:
                # Условие Армихо не выполнено для alpha=1
                alpha = 0.5
                while phi(alpha) > phi_0 + c1 * alpha * phi_grad_0:
                    alpha /= 2

        elif method == 'Armijo':
            # Условие Армихо
            c = line_search_options.get('c1', 1e-4)
            alpha = 1.0
            while oracle.func(x_k + alpha * d_k) > f_k + c * alpha * grad_k.dot(d_k):
                alpha /= 2

        # Обновление точки
        x_k = x_k + alpha * d_k

    message = f"iterations limit exceeded after {max_iter} iterations"
    return (x_k, message, history) if trace else (x_k, message)

#1.3
import numpy as np
from scipy.sparse import issparse
from scipy.special import expit
import scipy.sparse as sp

class LogRegL2Oracle:
    """Оракул для задачи двухклассовой логистической регрессии с L2-регуляризацией."""

    def __init__(self, A, b, regcoef):
        """
        Инициализация оракула.
        """
        self.A = A
        self.b = b
        self.regcoef = regcoef
        self.m = A.shape[0]
        self.n = A.shape[1]

    def func(self, x):
        """Значение функции в точке x."""
        # Вычисляем Ax (эффективно для плотных и разреженных матриц)
        if issparse(self.A):
            Ax = self.A.dot(x)
        else:
            Ax = self.A @ x

        # Вычисляем b_i * <a_i, x>
        bAx = self.b * Ax

        # Логистическая функция потерь
        # log(1 + exp(-z)) вычисляем через специальную функцию
        log_loss = np.logaddexp(0, -bAx).mean()

        # L2 регуляризация
        regularization = 0.5 * self.regcoef * np.dot(x, x)

        return log_loss + regularization

    def grad(self, x):
        """Градиент функции в точке x."""
        # Вычисляем Ax
        if issparse(self.A):
            Ax = self.A.dot(x)
        else:
            Ax = self.A @ x

        # Вычисляем sigma(-b_i * <a_i, x>) = 1 / (1 + exp(b_i * <a_i, x>))
        # что равно expit(-b_i * <a_i, x>)
        bAx = self.b * Ax
        sigma = expit(-bAx)

        # Градиент логистической функции потерь
        if issparse(self.A):
            # Для разреженной матрицы используем специальный метод
            grad_log_loss = -self.A.T.dot((sigma * self.b) / self.m)
        else:
            grad_log_loss = -self.A.T @ (sigma * self.b) / self.m

        # Градиент регуляризации
        grad_reg = self.regcoef * x

        return grad_log_loss + grad_reg

    def hess(self, x):
        """Гессиан функции в точке x."""
        # Вычисляем Ax
        if issparse(self.A):
            Ax = self.A.dot(x)
        else:
            Ax = self.A @ x

        # Вычисляем sigma(-b_i * <a_i, x>)
        bAx = self.b * Ax
        sigma = expit(-bAx)

        # Вычисляем диагональную матрицу D = diag(sigma * (1 - sigma))
        D = sigma * (1 - sigma) / self.m

        if issparse(self.A):
            # Для разреженной матрицы: A^T * D * A
            # Эффективное вычисление через поэлементное умножение
            A_scaled = self.A.multiply(np.sqrt(D)[:, np.newaxis])
            hess_log_loss = A_scaled.T.dot(A_scaled)

            # Добавляем регуляризацию
            hess_reg = self.regcoef * sp.eye(self.n, format='csr')
            return hess_log_loss + hess_reg
        else:
            # Для плотной матрицы
            hess_log_loss = self.A.T @ (D[:, np.newaxis] * self.A)
            hess_reg = self.regcoef * np.eye(self.n)
            return hess_log_loss + hess_reg

    def func_directional(self, x, d, alpha):
        """Значение функции вдоль направления d."""
        return self.func(x + alpha * d)

    def grad_directional(self, x, d, alpha):
        """Производная по направлению d в точке x + alpha*d."""
        point = x + alpha * d
        grad = self.grad(point)
        return np.dot(grad, d)


def create_log_reg_oracle(A, b, regcoef):
    """Создает оракул для логистической регрессии."""
    return LogRegL2Oracle(A, b, regcoef)

#1.4
def grad_finite_diff(oracle, x, eps=1e-7):
    """Вычисление градиента методом конечных разностей."""
    n = len(x)
    grad = np.zeros(n)

    for i in range(n):
        e_i = np.zeros(n)
        e_i[i] = 1.0

        f_plus = oracle.func(x + eps * e_i)
        f_minus = oracle.func(x - eps * e_i)

        grad[i] = (f_plus - f_minus) / (2 * eps)

    return grad

def hess_finite_diff(oracle, x, eps=1e-5):
    """Вычисление гессиана методом конечных разностей."""
    n = len(x)
    hess = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            e_i = np.zeros(n)
            e_i[i] = 1.0
            e_j = np.zeros(n)
            e_j[j] = 1.0

            # Формула для второй производной
            f_xx = oracle.func(x + eps*e_i + eps*e_j)
            f_x = oracle.func(x + eps*e_i)
            f_y = oracle.func(x + eps*e_j)
            f_0 = oracle.func(x)

            hess[i, j] = (f_xx - f_x - f_y + f_0) / (eps * eps)

            if i != j:
                hess[j, i] = hess[i, j]

    return hess

#1.5
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(('D:\1Programmirovanie\LLM универ\4 лаба\oracles.py'))

import oracles
from oracles import QuadraticOracle
from optimization import gradient_descent

# Создаем различные квадратичные функции
def create_quadratic_function(condition_number=1, rotation=0):
    """Создает квадратичную функцию с заданным числом обусловленности."""
    n = 2

    if condition_number == 1:
        # Идеально обусловленная (круглые линии уровня)
        A = np.eye(n)
    else:
        # Матрица с заданным числом обусловленности
        A = np.array([[condition_number, 0], [0, 1]])

        # Применяем вращение
        if rotation != 0:
            theta = rotation
            R = np.array([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])
            A = R @ A @ R.T

    b = np.zeros(n)
    return QuadraticOracle(A, b)

# Функция для рисования траектории
def plot_trajectory(oracle, x0, method='gradient', ax=None, **kwargs):
    """Рисует траекторию метода оптимизации."""
    if method == 'gradient':
        x_opt, message, history = gradient_descent(
            oracle, x0, tolerance=1e-6, max_iter=1000, trace=True
        )
    # ... можно добавить другие методы

    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    # Рисуем линии уровня
    x_range = np.linspace(-3, 3, 100)
    y_range = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x_range, y_range)

    Z = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = oracle.func(np.array([X[i, j], Y[i, j]]))

    ax.contour(X, Y, Z, levels=20, alpha=0.5)

    # Рисуем траекторию
    traj = np.array(history['x'])
    ax.plot(traj[:, 0], traj[:, 1], 'ro-', linewidth=2, markersize=4)
    ax.plot(x0[0], x0[1], 'go', markersize=10, label='Start')
    ax.plot(x_opt[0], x_opt[1], 'b*', markersize=15, label='Optimum')

    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.legend()
    ax.grid(True, alpha=0.3)

    return ax

# Запускаем эксперимент
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Разные числа обусловленности
condition_numbers = [1, 10, 100]
start_points = [np.array([2.5, 2.5]), np.array([-2, 2]), np.array([2, -2])]

for i, cond in enumerate(condition_numbers):
    for j, x0 in enumerate(start_points):
        oracle = create_quadratic_function(cond)
        ax = axes[j, i]
        plot_trajectory(oracle, x0, ax=ax)
        ax.set_title(f'Condition number = {cond}, Start = {x0}')
        ax.axis('equal')

plt.tight_layout()
plt.show()

#2.1
import numpy as np
import matplotlib.pyplot as plt
from oracles import QuadraticOracle
from optimization import gradient_descent

# Создаем различные квадратичные функции
def create_quadratic_function(condition_number=1, rotation=0):
    """Создает квадратичную функцию с заданным числом обусловленности."""
    n = 2

    if condition_number == 1:
        # Идеально обусловленная (круглые линии уровня)
        A = np.eye(n)
    else:
        # Матрица с заданным числом обусловленности
        A = np.array([[condition_number, 0], [0, 1]])

        # Применяем вращение
        if rotation != 0:
            theta = rotation
            R = np.array([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])
            A = R @ A @ R.T

    b = np.zeros(n)
    return QuadraticOracle(A, b)

# Функция для рисования траектории
def plot_trajectory(oracle, x0, method='gradient', ax=None, **kwargs):
    """Рисует траекторию метода оптимизации."""
    if method == 'gradient':
        x_opt, message, history = gradient_descent(
            oracle, x0, tolerance=1e-6, max_iter=1000, trace=True
        )
    # ... можно добавить другие методы

    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    # Рисуем линии уровня
    x_range = np.linspace(-3, 3, 100)
    y_range = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x_range, y_range)

    Z = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = oracle.func(np.array([X[i, j], Y[i, j]]))

    ax.contour(X, Y, Z, levels=20, alpha=0.5)

    # Рисуем траекторию
    traj = np.array(history['x'])
    ax.plot(traj[:, 0], traj[:, 1], 'ro-', linewidth=2, markersize=4)
    ax.plot(x0[0], x0[1], 'go', markersize=10, label='Start')
    ax.plot(x_opt[0], x_opt[1], 'b*', markersize=15, label='Optimum')

    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.legend()
    ax.grid(True, alpha=0.3)

    return ax

# Запускаем эксперимент
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Разные числа обусловленности
condition_numbers = [1, 10, 100]
start_points = [np.array([2.5, 2.5]), np.array([-2, 2]), np.array([2, -2])]

for i, cond in enumerate(condition_numbers):
    for j, x0 in enumerate(start_points):
        oracle = create_quadratic_function(cond)
        ax = axes[j, i]
        plot_trajectory(oracle, x0, ax=ax)
        ax.set_title(f'Condition number = {cond}, Start = {x0}')
        ax.axis('equal')

plt.tight_layout()
plt.show()

#2.2
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags

def run_condition_number_experiment(n_values=[10, 100, 1000],
                                    kappa_values=np.logspace(0, 3, 20),
                                    n_trials=5):
    """Исследует зависимость числа итераций от числа обусловленности."""

    results = {n: [] for n in n_values}

    for n in n_values:
        print(f"Running experiments for n={n}...")

        for trial in range(n_trials):
            iterations_vs_kappa = []

            for kappa in kappa_values:
                # Генерируем диагональную матрицу с заданным числом обусловленности
                # Диагональные элементы от 1 до kappa
                diag_vals = np.linspace(1, kappa, n)
                A = diags(diag_vals, format='csr')
                b = np.random.randn(n)

                oracle = QuadraticOracle(A, b)
                x0 = np.random.randn(n)

                # Запускаем градиентный спуск
                x_opt, message, history = gradient_descent(
                    oracle, x0, tolerance=1e-6, max_iter=10000, trace=True
                )

                iterations_vs_kappa.append(len(history['func']))

            results[n].append(iterations_vs_kappa)

    # Визуализация результатов
    colors = ['r', 'b', 'g', 'm', 'c']

    plt.figure(figsize=(12, 8))

    for i, n in enumerate(n_values):
        color = colors[i % len(colors)]

        for trial_data in results[n]:
            plt.loglog(kappa_values, trial_data, color=color, alpha=0.3, linewidth=1)

        # Среднее значение
        mean_iterations = np.mean(results[n], axis=0)
        plt.loglog(kappa_values, mean_iterations, color=color,
                  linewidth=3, label=f'n={n}')

    plt.xlabel('Condition number (κ)', fontsize=14)
    plt.ylabel('Number of iterations', fontsize=14)
    plt.title('Dependence of GD iterations on condition number', fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.show()

    return results

# Запускаем эксперимент
results = run_condition_number_experiment()

#2.3
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.datasets import load_svmlight_file
from oracles import create_log_reg_oracle
from optimization import gradient_descent, newton

def compare_methods_on_real_data(dataset_name='w8a'):
    """Сравнивает градиентный спуск и метод Ньютона на реальных данных."""

    # Загружаем данные
    if dataset_name == 'w8a':
        A, b = load_svmlight_file('data/w8a')
    elif dataset_name == 'gisette':
        A, b = load_svmlight_file('data/gisette_scale')
    elif dataset_name == 'real-sim':
        A, b = load_svmlight_file('data/real-sim')

    # Нормализуем метки к {-1, 1}
    b = np.where(b > 0, 1, -1)

    # Параметры
    m, n = A.shape
    regcoef = 1.0 / m
    x0 = np.zeros(n)

    # Создаем оракул
    oracle = create_log_reg_oracle(A, b, regcoef)

    # Запускаем градиентный спуск
    print(f"Running Gradient Descent on {dataset_name}...")
    start_time = time.time()
    x_gd, msg_gd, history_gd = gradient_descent(
        oracle, x0, tolerance=1e-6, max_iter=1000, trace=True
    )
    gd_time = time.time() - start_time

    # Запускаем метод Ньютона
    print(f"Running Newton's method on {dataset_name}...")
    start_time = time.time()
    x_newton, msg_newton, history_newton = newton(
        oracle, x0, tolerance=1e-6, max_iter=100, trace=True
    )
    newton_time = time.time() - start_time

    print(f"\nResults for {dataset_name}:")
    print(f"GD: {msg_gd}, time: {gd_time:.2f}s")
    print(f"Newton: {msg_newton}, time: {newton_time:.2f}s")

    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # График 1: Значение функции vs время
    axes[0].plot(history_gd['func'], 'b-', linewidth=2, label='Gradient Descent')
    axes[0].plot(history_newton['func'], 'r-', linewidth=2, label="Newton's method")
    axes[0].set_xlabel('Iteration', fontsize=12)
    axes[0].set_ylabel('Function value', fontsize=12)
    axes[0].set_title(f'Convergence: {dataset_name}', fontsize=14)
    axes[0].legend(fontsize=12)
    axes[0].grid(True, alpha=0.3)

    # График 2: Норма градиента vs время
    grad_norm_0 = np.linalg.norm(oracle.grad(x0))

    # Для градиентного спуска
    gd_grad_norm_rel = [gn / grad_norm_0 for gn in history_gd['grad_norm']]

    # Для метода Ньютона
    newton_grad_norm_rel = [gn / grad_norm_0 for gn in history_newton['grad_norm']]

    axes[1].semilogy(gd_grad_norm_rel, 'b-', linewidth=2, label='Gradient Descent')
    axes[1].semilogy(newton_grad_norm_rel, 'r-', linewidth=2, label="Newton's method")
    axes[1].set_xlabel('Iteration', fontsize=12)
    axes[1].set_ylabel('Relative gradient norm', fontsize=12)
    axes[1].set_title(f'Gradient norm reduction: {dataset_name}', fontsize=14)
    axes[1].legend(fontsize=12)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Анализ сложности
    print("\nComplexity analysis:")
    print(f"Data dimensions: m={m}, n={n}")
    print(f"\nGradient Descent per iteration:")
    print(f"  - Computation: O(m*n) for Ax")
    print(f"  - Memory: O(m + n)")

    print(f"\nNewton's method per iteration:")
    print(f"  - Computation: O(m*n^2 + n^3) for Hessian and solve")
    print(f"  - Memory: O(n^2) for Hessian")

    return history_gd, history_newton

# Запускаем сравнение для разных датасетов
datasets = ['w8a', 'gisette', 'real-sim']
for dataset in datasets:
    try:
        compare_methods_on_real_data(dataset)
    except FileNotFoundError:
        print(f"Dataset {dataset} not found. Skipping...")



<>:330: SyntaxWarning: invalid escape sequence '\L'
<>:330: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Nikolaii\AppData\Local\Temp\ipykernel_8000\140941779.py:330: SyntaxWarning: invalid escape sequence '\L'
  sys.path.append(('D:\1Programmirovanie\LLM универ\4 лаба\oracles.py'))
C:\Users\Nikolaii\AppData\Local\Temp\ipykernel_8000\140941779.py:330: SyntaxWarning: invalid escape sequence '\L'
  sys.path.append(('D:\1Programmirovanie\LLM универ\4 лаба\oracles.py'))


ModuleNotFoundError: No module named 'oracles'